# Setting

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!apt-get update -qq # 시스템에서 패키지 설치시 사용되는 명령어, 모든 패키지를 최신버전으로 업그레이드
!apt-get install fonts-nanum* -qq

In [26]:
import matplotlib.font_manager as fm
fm._rebuild()
sys_font = fm.findSystemFonts()
[f for f in sys_font if 'Nanum' in f]

['/usr/share/fonts/truetype/nanum/NanumMyeongjoEcoExtraBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBrush.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothic.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicEcoBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicEcoExtraBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareEB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunpenR.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicExtraBold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumSquareRoundEB.ttf',
 '/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf',
 '/usr/share/fonts/truetype/nanum/NanumGothicCoding-Bold.ttf',
 '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf',
 '/usr/share/fon

In [27]:
import matplotlib.pyplot as plt

path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
fm._rebuild()

NanumGothic


In [28]:
#한글폰트 설치 확인
for fontInfo in fm.fontManager.ttflist: 
  if 'Nanum' in fontInfo.name:
    print(fontInfo.name+"="+fontInfo.fname)

NanumMyeongjo Eco=/usr/share/fonts/truetype/nanum/NanumMyeongjoEcoExtraBold.ttf
NanumSquareRound=/usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf
Nanum Brush Script=/usr/share/fonts/truetype/nanum/NanumBrush.ttf
NanumGothic=/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf
NanumGothic=/usr/share/fonts/truetype/nanum/NanumGothic.ttf
NanumGothic Eco=/usr/share/fonts/truetype/nanum/NanumGothicEcoBold.ttf
NanumGothic Eco=/usr/share/fonts/truetype/nanum/NanumGothicEcoExtraBold.ttf
NanumSquareRound=/usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf
NanumSquare=/usr/share/fonts/truetype/nanum/NanumSquareEB.ttf
NanumGothic Eco=/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf
NanumMyeongjo=/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf
NanumBarunpen=/usr/share/fonts/truetype/nanum/NanumBarunpenR.ttf
NanumGothic=/usr/share/fonts/truetype/nanum/NanumGothicExtraBold.ttf
NanumSquareRound=/usr/share/fonts/truetype/nanum/NanumSquareRoundEB.ttf
NanumMyeongjo=/usr/share/fonts/truetyp

In [29]:
!pip install konlpy wordcloud

In [30]:
import pandas as pd
import seaborn as sns
import numpy as np

import nltk
from konlpy.tag import Okt
from tqdm import tqdm
from collections import OrderedDict, defaultdict

In [31]:
## install Mecab

# !apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

from konlpy.tag import Mecab
from collections import Counter

mec = Mecab() # 런타임 다시 연결하기

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
openjdk-8-jdk is already the newest version (8u292-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [77]:
df = pd.read_excel('/content/drive/MyDrive/Codepresso/TeamProject/data/combined_train.xlsx')
df = df[['hotel', 'separate_review', 'sentiment']]

df.isnull().sum()
df = df.dropna() # 17649개
df = df.reset_index()
df = df.drop(columns='index')
df

,hotel,separate_review,sentiment
0,더 베스트 제주 성산(구 라마다 앙코르 성산),직원분들 친절하고 좋았고요 가격은 싼 거 같 조식도 깔끔했고요,1.0
1,더 베스트 제주 성산(구 라마다 앙코르 성산),호텔이 낡아서 그런지 카펫과 에어컨에서 쾌쾌한 냄새가 많이 났고 천장 환풍기에 까...,0.0
2,더 베스트 제주 성산(구 라마다 앙코르 성산),샤워실 바닥 유리 쪽에도 까만 곰팡이가 많이 보였,0.0
3,더 베스트 제주 성산(구 라마다 앙코르 성산),커튼도 많이 더러웠고 방충망에 구멍이 뚫려 있어서 제가 반창고로 막아서 사용했,0.0
4,더 베스트 제주 성산(구 라마다 앙코르 성산),침대에 배드 버그가 있는지 물리고 간지러워 잠을 거의 못 잤,0.0
...,...,...,...
17644,휘슬락 호텔,와이파이랑 다시 보기가 없,1.0
17645,휘슬락 호텔,오오 엄청 깔끔하고 바다 경치 진짜 좋,1.0
17646,히든 클리프 호텔 앤 네이쳐,직원분들도 친절하시고요 와이파이는 설치해주세요 오 산꼭대기 목장에도 와이파이가 있는데,1.0
17647,히든 클리프 호텔 앤 네이쳐,숙소도 깨끗하고 고급스럽고 수영장 물 온도도 적당하고 너무 예뻐요 조식도 맛있어요 ...,1.0


In [78]:
from itertools import product

# sentiment = 긍정 리뷰
positive = df[df['sentiment'] == 1]
positive = positive[['hotel','separate_review','sentiment']]

# sentiment = 부정 리뷰
negative = df[df['sentiment'] == 0]
negative = negative[['hotel','separate_review','sentiment']]

print(positive.shape,negative.shape)

positive = positive.reset_index()
positive = positive.drop(columns='index')
negative = negative.reset_index()
negative = negative.drop(columns='index')

positive # 14:3

(14244, 3) (3258, 3)


,hotel,separate_review,sentiment
0,더 베스트 제주 성산(구 라마다 앙코르 성산),직원분들 친절하고 좋았고요 가격은 싼 거 같 조식도 깔끔했고요,1.0
1,더 베스트 제주 성산(구 라마다 앙코르 성산),룸 컨디션은 괜찮은 편이었지만,1.0
2,더 베스트 제주 성산(구 라마다 앙코르 성산),숙소 청결함,1.0
3,더 포 그레이스 리조트,싼 맛에 있긴 했지만,1.0
4,라마다 제주시티 호텔,뭐 근처가 다 오래되고 갈만한 곳이 없으니 여기가 잴 무난하니까요,1.0
...,...,...,...
14239,휘슬락 호텔,와이파이랑 다시 보기가 없,1.0
14240,휘슬락 호텔,오오 엄청 깔끔하고 바다 경치 진짜 좋,1.0
14241,히든 클리프 호텔 앤 네이쳐,직원분들도 친절하시고요 와이파이는 설치해주세요 오 산꼭대기 목장에도 와이파이가 있는데,1.0
14242,히든 클리프 호텔 앤 네이쳐,숙소도 깨끗하고 고급스럽고 수영장 물 온도도 적당하고 너무 예뻐요 조식도 맛있어요 ...,1.0


In [79]:
df = df.groupby('hotel')['separate_review'].apply(list).to_dict()

# key : 호텔명 가져오기
hotel_list = list(df.keys())
hotel_list
print(len(hotel_list))

In [ ]:
# hotel_list = ['더 베스트 제주 성산(구 라마다 앙코르 성산)',
#  '더 포 그레이스 리조트',
#  '더큐브 리조트 제주',
#  '라마다 제주시티 호텔',
#  '랜딩관 제주신화월드 호텔앤리조트',
#  '롯데 시티호텔 제주',
#  '베스트웨스턴 제주',
#  '봄그리고가을 호텔&리조트',
#  '브라운 스위트 제주',
#  '신화관 제주 신화월드 호텔앤리조트',
#  '아트스테이 서귀포 하버',
#  '에벤에셀 제주 함덕호텔',
#  '윈스토리 호텔 서귀포',
#  '유탑유블레스호텔 제주함덕',
#  '제주 부영호텔&리조트',
#  '제주 센트럴 시티 호텔',
#  '제주 헤이, 서귀포',
#  '코델리아s 호텔 앤 리조트',
#  '코업시티호텔 성산',
#  '팜파스호텔 제주',
#  '호텔 골든데이지 서귀포 오션',
#  '호텔 리젠트마린',
#  '호텔 시리우스 제주',
#  '호텔 앨리스 앤 트렁크',
#  '호텔 에어시티 제주',
#  '호텔 컬리넌 제주',
#  '호텔 케니 서귀포',
#  '호텔샬롬 제주',
#  '호텔위드 제주',
#  '휘슬락 호텔',
#  '히든 클리프 호텔 앤 네이쳐']

In [82]:
# 데이터셋에 맞는 불용어 리스트 생성
stopwords = ['굿', '아침', '저녁', '제주도', '제주', '호텔', '리뷰', '숙소', '여행', '트립',
             '한라산', '우도', '성산', '서귀포', '올레', '라마', '진짜', '바로', '함덕', '다시',
             '다른', '다음', '보기', '정도', '이중섭', '방이', '완전', '정도', '정말',
             '다만', '최고', '아주', '역시', '조금', '매우', '모두', '물이', '박이',
             '처음', '무엇', '계속', '제대로', '묵고', '하루', '일출봉', '성산일출봉', '성산', '그냥',
             '별로', '사용', '단위', '달라', '여기', '집도', '방문', '요도', '때문', '생각',
             '하나', '마지막', '보이', '느낌', '확인', '모든', '에아', '천지연폭포', '저기', '보고',
             '간다', '방도', '해동', '중문', '부영', '그거', '동안', '자주', '이번', '저번',
             '부터', '비도', '가면', '식도', '제일', '고민', '월드', '약간', '미리', '가장',
             '기대', '해주시', '그것', '조절', '동문', '가기', '걱정', '한번', '예전', '그대로', 
             '신경', '나름', '가기', '크게', '걱정', '전체', '그것', '추가', '살짝', '또한',
             '기분', '부분', '커서', '항상', '준비', '얼마', '경험', '쓰기', '이용', '너무', 
             '신화', '많이', '그런지', '엄청', '보다', '같이', '굉장히', '산동', '가능',
             '전혀', '에서', '오래', '힘들', '슴벅', '없이', '해결', '이틀', '시간', '요수']

In [83]:
positive = positive.groupby('hotel')['separate_review'].apply(list).to_dict()
negative = negative.groupby('hotel')['separate_review'].apply(list).to_dict()

In [84]:
# 불용어 적용
for i in range(len(df)):
  for j in range(len(positive[hotel_list[i]])):
    for s in stopwords:
      positive[hotel_list[i]][j] = positive[hotel_list[i]][j].replace(s, '')

for i in range(len(negative)):
  for j in range(len(negative[hotel_list[i]])):
    for s in stopwords:
      negative[hotel_list[i]][j] = negative[hotel_list[i]][j].replace(s, '')

# 긍정 - 형태소 분석 + 다빈도 단어 추출

In [85]:
# 호텔별 형태소 분석 후 다빈도 단어를 넣을 리스트 - 긍정
pos_reviews = []

for i in range(len(positive)):
  # 형태소 추출 리스트
  pos_list = []

  # 말뭉치(corpus) 생성
  corpus = "".join(positive[hotel_list[i]])
  # corpus에서 형태소 추출
  mec_pos = mec.pos(corpus)

  # 키워드 추출을 위해서는 pos='NNG', 'VA', XR', MAG의 빈도를 확인해야 함
  for i in range(len(mec_pos)):
    if 'NNG' in mec_pos[i][1]:
      pos_list.append(mec_pos[i][0])
    if 'VA' in mec_pos[i][1]:
      pos_list.append(mec_pos[i][0])
    if 'XR' in mec_pos[i][1]:
      pos_list.append(mec_pos[i][0])
    if 'MAG' in mec_pos[i][1]:
      pos_list.append(mec_pos[i][0])

  # 다빈도 단어 추출
  counter = Counter(pos_list)
  # 가장 높은 빈도 단어 50개 출력
  counter.most_common(50)
  # 한글자 명사 제거
  available_counter = Counter({x: counter[x] for x in counter if len(x) > 1 })
  common_noun = available_counter.most_common(50)
  pos_reviews.append(common_noun)

pos_reviews

[[('친절', 37),
  ('경치', 31),
  ('깨끗', 26),
  ('직원', 22),
  ('괜찮', 22),
  ('만족', 22),
  ('깔끔', 21),
  ('객실', 16),
  ('주차', 16),
  ('가까워서', 14),
  ('저렴', 14),
  ('구성비', 13),
  ('편의점', 13),
  ('위치', 13),
  ('시설', 13),
  ('주차장', 13),
  ('조식', 12),
  ('가격', 11),
  ('근처', 10),
  ('공간', 9),
  ('수영장', 8),
  ('주변', 8),
  ('침대', 7),
  ('예약', 7),
  ('야외', 7),
  ('청소', 7),
  ('불편', 7),
  ('청결', 6),
  ('일출', 6),
  ('감사', 6),
  ('추천', 6),
  ('체크인', 6),
  ('컨디션', 5),
  ('편하', 5),
  ('가깝', 5),
  ('프런트', 5),
  ('맛있', 5),
  ('가까운', 4),
  ('지하', 4),
  ('루프', 4),
  ('업그레이드', 4),
  ('아기', 4),
  ('함께', 4),
  ('구비', 4),
  ('에어컨', 4),
  ('식당', 4),
  ('옥상', 4),
  ('따로', 4),
  ('베스트', 4),
  ('건물', 4)],
 [('수영장', 6),
  ('깨끗', 5),
  ('가깝', 4),
  ('가족', 3),
  ('리조트', 3),
  ('깔끔', 3),
  ('저렴', 3),
  ('가격', 3),
  ('직원', 2),
  ('친절', 2),
  ('편의점', 2),
  ('산책로', 2),
  ('바다', 2),
  ('재의', 2),
  ('위치', 2),
  ('문제', 1),
  ('커플', 1),
  ('시설', 1),
  ('서비스', 1),
  ('근접', 1),
  ('객실', 1),
  ('실내', 1),
  ('충전', 1)],
 [('깨끗', 6

In [86]:
# dataframe으로 두 리스트 (호텔명 + 다빈도 단어)를 합치기
pos_final_df = pd.DataFrame([x for x in zip(hotel_list, pos_reviews)])
# index명으로 된 칼럼 이름을 새로 지정
mapping = {pos_final_df.columns[0]: 'hotel_name', pos_final_df.columns[1]: 'noun_extraction'}
pos_final_df = pos_final_df.rename(columns=mapping)

,hotel_name,noun_extraction
0,더 베스트 제주 성산(구 라마다 앙코르 성산),"[(친절, 37), (경치, 31), (깨끗, 26), (직원, 22), (괜찮, ..."
1,더 포 그레이스 리조트,"[(수영장, 6), (깨끗, 5), (가깝, 4), (가족, 3), (리조트, 3)..."
2,라마다 제주시티 호텔,"[(깨끗, 68), (깔끔, 52), (공항, 43), (친절, 40), (위치, ..."
3,랜딩관 제주신화월드 호텔앤리조트,"[(친절, 29), (시설, 27), (깨끗, 22), (깔끔, 19), (만족, ..."
4,롯데 시티호텔 제주,"[(친절, 43), (공항, 43), (수영장, 41), (깨끗, 24), (경치,..."
5,베스트웨스턴 제주,"[(공항, 43), (깔끔, 27), (깨끗, 26), (위치, 25), (시설, ..."
6,봄그리고가을 호텔&리조트,"[(경치, 285), (바다, 141), (깔끔, 133), (깨끗, 133), (..."
7,브라운 스위트 제주,"[(경치, 41), (깔끔, 26), (깨끗, 26), (위치, 24), (친절, ..."
8,블루하와이호텔,"[(구성비, 63), (친절, 48), (수영장, 42), (경치, 40), (깨끗..."
9,서귀포 라마다 앙코르 호텔,"[(구성비, 49), (괜찮, 45), (친절, 42), (가격, 37), (깨끗,..."


In [87]:
pos_final_df.to_csv('/content/drive/MyDrive/Codepresso/TeamProject/data/hotel_review200_pos_noun_extraction.csv')

# 부정 - 형태소 분석 + 다빈도 단어 추출

In [ ]:
# 호텔별 형태소 분석 후 다빈도 단어를 넣을 리스트 - 긍정
neg_reviews = []

for i in range(len(negative)):
  # 형태소 추출 리스트
  pos_list = []

  # 말뭉치(corpus) 생성
  corpus = "".join(negative[hotel_list[i]])
  # corpus에서 형태소 추출
  mec_pos = mec.pos(corpus)

  # 키워드 추출을 위해서는 pos='NNG', 'VA', XR', MAG의 빈도를 확인해야 함
  for i in range(len(mec_pos)):
    if 'NNG' in mec_pos[i][1]:
      pos_list.append(mec_pos[i][0])
    if 'VA' in mec_pos[i][1]:
      pos_list.append(mec_pos[i][0])
    if 'XR' in mec_pos[i][1]:
      pos_list.append(mec_pos[i][0])
    if 'MAG' in mec_pos[i][1]:
      pos_list.append(mec_pos[i][0])

  # 다빈도 단어 추출
  counter = Counter(pos_list)
  # 가장 높은 빈도 단어 50개 출력
  counter.most_common(50)
  # 한글자 명사 제거
  available_counter = Counter({x: counter[x] for x in counter if len(x) > 1 })
  common_noun = available_counter.most_common(50)
  neg_reviews.append(common_noun)


In [ ]:
# dataframe으로 두 리스트 (호텔명 + 다빈도 단어)를 합치기
neg_final_df = pd.DataFrame([x for x in zip(hotel_list, neg_reviews)])
# index명으로 된 칼럼 이름을 새로 지정
mapping = {neg_final_df.columns[0]: 'hotel_name', neg_final_df.columns[1]: 'noun_extraction'}
neg_final_df = neg_final_df.rename(columns=mapping)

In [91]:
neg_final_df.to_csv('/content/drive/MyDrive/Codepresso/TeamProject/data/hotel_review200_neg_noun_extraction.csv')

# WordCloud

## 긍정

In [93]:
pos_df = pd.read_csv('/content/drive/MyDrive/Codepresso/TeamProject/data/hotel_review200_pos_noun_extraction.csv')
pos_df = pos_df.drop([pos_df.columns[0]], axis='columns')
pos_df

,hotel_name,noun_extraction
0,더 베스트 제주 성산(구 라마다 앙코르 성산),"[('친절', 37), ('경치', 31), ('깨끗', 26), ('직원', 22..."
1,더 포 그레이스 리조트,"[('수영장', 6), ('깨끗', 5), ('가깝', 4), ('가족', 3), ..."
2,라마다 제주시티 호텔,"[('깨끗', 68), ('깔끔', 52), ('공항', 43), ('친절', 40..."
3,랜딩관 제주신화월드 호텔앤리조트,"[('친절', 29), ('시설', 27), ('깨끗', 22), ('깔끔', 19..."
4,롯데 시티호텔 제주,"[('친절', 43), ('공항', 43), ('수영장', 41), ('깨끗', 2..."
5,베스트웨스턴 제주,"[('공항', 43), ('깔끔', 27), ('깨끗', 26), ('위치', 25..."
6,봄그리고가을 호텔&리조트,"[('경치', 285), ('바다', 141), ('깔끔', 133), ('깨끗',..."
7,브라운 스위트 제주,"[('경치', 41), ('깔끔', 26), ('깨끗', 26), ('위치', 24..."
8,블루하와이호텔,"[('구성비', 63), ('친절', 48), ('수영장', 42), ('경치', ..."
9,서귀포 라마다 앙코르 호텔,"[('구성비', 49), ('괜찮', 45), ('친절', 42), ('가격', 3..."


### noun_extraction column 전처리 + 형태 변환

모든 특수문자, 마침표, 쉼표, 숫자, 한글자 씩 인덱스화 된 것을 전처리하기

In [94]:
pos_w_list = []

for i in range(0, len(pos_df)):
  # 전처리
  s = pos_df['noun_extraction'][i].replace("(", "")
  s = s.replace(")", "")
  s = s.replace("[", "")
  s = s.replace("]", "")
  s = s.replace("'", "")
  s = s.replace(" ", "")
  # 쉼표를 중심으로 문자열 단어와 문자열 숫자 나누기
  s = s.split(',')
  # 짝수 인덱스 -> 단어  |  홀수 인덱스 -> 빈도로 나누기
  word = s[0::2]
  freq = s[1::2]
  # 문자열 숫자를 실수형으로 바꾸기
  freq = list(map(int,freq))
  # 모두 빈 리스트에 넣어주기
  pos_w_list.append(word)
  pos_w_list.append(freq)

pos_w_list에서 짝수 인덱스는 단어 리스트에, 홀수 인덱스는 빈도 리스트에 저장

In [95]:
# 단어와 빈도 각각 나눠서 변수 지정
pos_word_list = pos_w_list[0::2]
pos_freq_list = pos_w_list[1::2]

### visualization

In [96]:
## Setting

# colab 한글폰트 깨짐 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from numpy import array
!pip install wordcloud
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()

matplotlib.rc('font', family=font_name)
# %config InlineBackend.figure_format = 'retina' # 선명하게 출력

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 4 dirs
/usr/share/fonts/truetype/dejavu: caching, new cache contents: 22 fonts, 0 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 31 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existen

워드클라우드 생성 시, 딕셔너리만 사용 가능하기 때문에 단어-빈도의 쌍을 딕셔너리 형태로 변환

In [97]:
# for loop으로 새로운 딕셔너리 30개 생성
pos_dic = {}

for i in range(0, len(pos_df)):
  pos_dic[i] = {}

In [ ]:
for i in tqdm(range(0, len(pos_df))):
  # 31개 빈 딕셔너리에 합친 두 리스트 key: value로 저장
  pos_dic[i] = {name:value for name, value in zip(pos_word_list[i], pos_freq_list[i])}

# 구성비 -> 가성비로 교체
for i in range(0, len(pos_df)):
  if '구성비' in pos_dic[i]:
    pos_dic[i]['가성비'] = pos_dic[i].pop('구성비')


In [100]:
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt   
%matplotlib inline
from IPython.display import set_matplotlib_formats
matplotlib.rc('font',family = 'NanumGothic')
set_matplotlib_formats('retina')
matplotlib.rc('axes',unicode_minus = False)

# 호텔별 워드클라우드 생성 
for i in tqdm(range(0, len(pos_dic))):
  wc = WordCloud(font_path ='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', background_color='white',
                 random_state=None, colormap = "Accent_r", width=1500, height=1000).generate_from_frequencies(pos_dic[i])
  plt.figure(1, figsize=(15,10))
  plt.imshow(wc)
  plt.axis('off')
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

## 부정

In [101]:
neg_df = pd.read_csv('/content/drive/MyDrive/Codepresso/TeamProject/data/hotel_review200_neg_noun_extraction.csv')
# 카피본 만들기
neg_df = neg_df.drop([neg_df.columns[0]], axis='columns')
neg_df

,hotel_name,noun_extraction
0,더 베스트 제주 성산(구 라마다 앙코르 성산),"[('경치', 6), ('바닥', 5), ('주차장', 5), ('욕실', 5), ..."
1,더 포 그레이스 리조트,"[('수영장', 5), ('요청', 5), ('청소', 4), ('가운', 4), ..."
2,라마다 제주시티 호텔,"[('침대', 20), ('불편', 13), ('냄새', 11), ('주차', 11..."
3,랜딩관 제주신화월드 호텔앤리조트,"[('객실', 6), ('위치', 5), ('서비스', 5), ('최악', 5), ..."
4,롯데 시티호텔 제주,"[('경치', 3), ('객실', 2), ('할인', 2), ('굳이', 2), (..."
5,베스트웨스턴 제주,"[('청소', 16), ('객실', 15), ('예약', 10), ('숙박', 10..."
6,봄그리고가을 호텔&리조트,"[('객실', 27), ('사진', 26), ('청소', 21), ('수영장', 2..."
7,브라운 스위트 제주,"[('주차', 15), ('화장실', 14), ('경치', 13), ('불편', 1..."
8,블루하와이호텔,"[('소리', 17), ('방음', 9), ('주차', 7), ('부족', 7), ..."
9,서귀포 라마다 앙코르 호텔,"[('불편', 15), ('화장실', 13), ('청소', 12), ('객실', 1..."


### noun_extraction column 전처리 + 형태 변환

In [102]:
neg_w_list = []

for i in range(0, len(neg_df)):
  # 전처리
  s = neg_df['noun_extraction'][i].replace("(", "")
  s = s.replace(")", "")
  s = s.replace("[", "")
  s = s.replace("]", "")
  s = s.replace("'", "")
  s = s.replace(" ", "")
  # 쉼표를 중심으로 문자열 단어와 문자열 숫자 나누기
  s = s.split(',')
  # 짝수 인덱스 -> 단어  |  홀수 인덱스 -> 빈도로 나누기
  word = s[0::2]
  freq = s[1::2]
  # 문자열 숫자를 실수형으로 바꾸기
  freq = list(map(int,freq))
  # 모두 빈 리스트에 넣어주기
  neg_w_list.append(word)
  neg_w_list.append(freq)

In [105]:
# 단어와 빈도 각각 나눠서 변수 지정
neg_word_list = neg_w_list[0::2]
neg_freq_list = neg_w_list[1::2]

### Visualization 

In [106]:
# for loop으로 새로운 딕셔너리 30개 생성
neg_dic = {}

for i in range(0, len(neg_df)):
  neg_dic[i] = {}

In [108]:
for i in tqdm(range(0, len(neg_df))):
  # 31개 빈 딕셔너리에 합친 두 리스트 key: value로 저장
  neg_dic[i] = {name:value for name, value in zip(neg_word_list[i], neg_freq_list[i])}

# 구성비 -> 가성비로 교체
for i in range(0, len(neg_df)):
  if '구성비' in neg_dic[i]:
    neg_dic[i]['가성비'] = neg_dic[i].pop('구성비')

100%|██████████| 39/39 [00:00<00:00, 28057.95it/s]


In [109]:
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt   
%matplotlib inline
from IPython.display import set_matplotlib_formats
matplotlib.rc('font',family = 'NanumGothic')
set_matplotlib_formats('retina')
matplotlib.rc('axes',unicode_minus = False)

# 호텔별 워드클라우드 생성 
for i in tqdm(range(0, len(neg_dic))):
  wc = WordCloud(font_path ='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', background_color='white',
                 random_state=None, colormap = "Accent_r", width=1500, height=1000).generate_from_frequencies(neg_dic[i])
  plt.figure(1, figsize=(15,10))
  plt.imshow(wc)
  plt.axis('off')
  plt.show()

Output hidden; open in https://colab.research.google.com to view.